---
sidebar_label: PDF文件

---

# WebPDFLoader

本笔记本提供了关于如何快速入门 [WebPDFLoader](/docs/integrations/document_loaders/) 的简要概述。如需查看所有 WebPDFLoader 功能和配置的详细文档，请前往 [API 参考文档](https://api.js.langchain.com/classes/langchain_community_document_loaders_web_pdf.WebPDFLoader.html)。

## 概览
### 集成详情

| 类 | 包 | 本地支持 | 可序列化 | Python 支持 |
| :--- | :--- | :---: | :---: |  :---: |
| [WebPDFLoader](https://api.js.langchain.com/classes/langchain_community_document_loaders_web_pdf.WebPDFLoader.html) | [@langchain/community](https://api.js.langchain.com/modules/langchain_community_document_loaders_web_pdf.html) | ✅ | beta | ❌ | 
### 加载器功能
| 来源 | 网页加载器 | 仅限 Node 环境 |
| :---: | :---: | :---: | 
| WebPDFLoader | ✅ | ❌ | 

你可以在网页环境中使用这个流行的 PDFLoader 版本。
默认情况下，PDF 文件中的每一页都会创建一个文档，你可以通过将 `splitPages` 选项设置为 `false` 来更改此行为。

## 准备

要使用 `WebPDFLoader` 文档加载器，你需要安装 `@langchain/community` 集成包以及 `pdf-parse` 包：

### 凭据

如果你想自动追踪模型调用，也可以取消下面的注释来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```bash
# export LANGSMITH_TRACING="true"
# export LANGSMITH_API_KEY="your-api-key"
```

### 安装

LangChain WebPDFLoader 集成位于 `@langchain/community` 包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core pdf-parse
</Npm2Yarn>

```

## 实例化

现在我们可以实例化我们的模型对象并加载文档：

In [1]:
import fs from "fs/promises";
import { WebPDFLoader } from "@langchain/community/document_loaders/web/pdf"

const nike10kPDFPath = "../../../../data/nke-10k-2023.pdf";

// Read the file as a buffer
const buffer = await fs.readFile(nike10kPDFPath);

// Create a Blob from the buffer
const nike10kPDFBlob = new Blob([buffer], { type: 'application/pdf' });

const loader = new WebPDFLoader(nike10kPDFBlob, {
  // required params = ...
  // optional params = ...
})

## 加载

In [2]:
const docs = await loader.load()
docs[0]

Document {
  pageContent: 'Table of Contents\n' +
    'UNITED STATES\n' +
    'SECURITIES AND EXCHANGE COMMISSION\n' +
    'Washington, D.C. 20549\n' +
    'FORM 10-K\n' +
    '(Mark One)\n' +
    '☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934\n' +
    'FOR THE FISCAL YEAR ENDED MAY 31, 2023\n' +
    'OR\n' +
    '☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934\n' +
    'FOR THE TRANSITION PERIOD FROM                         TO                         .\n' +
    'Commission File No. 1-10635\n' +
    'NIKE, Inc.\n' +
    '(Exact name of Registrant as specified in its charter)\n' +
    'Oregon93-0584541\n' +
    '(State or other jurisdiction of incorporation)(IRS Employer Identification No.)\n' +
    'One Bowerman Drive, Beaverton, Oregon 97005-6453\n' +
    '(Address of principal executive offices and zip code)\n' +
    '(503) 671-6453\n' +
    "(Registrant's telephone number, including area code)\n" +
 

In [3]:
console.log(docs[0].metadata)

{
  pdf: {
    version: '1.10.100',
    info: {
      PDFFormatVersion: '1.4',
      IsAcroFormPresent: false,
      IsXFAPresent: false,
      Title: '0000320187-23-000039',
      Author: 'EDGAR Online, a division of Donnelley Financial Solutions',
      Subject: 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31',
      Keywords: '0000320187-23-000039; ; 10-K',
      Creator: 'EDGAR Filing HTML Converter',
      Producer: 'EDGRpdf Service w/ EO.Pdf 22.0.40.0',
      CreationDate: "D:20230720162200-04'00'",
      ModDate: "D:20230720162208-04'00'"
    },
    metadata: null,
    totalPages: 107
  },
  loc: { pageNumber: 1 }
}


## 用法，自定义 `pdfjs` 构建

默认情况下，我们使用与 `pdf-parse` 捆绑在一起的 `pdfjs` 构建版本，它与大多数环境兼容，包括 Node.js 和现代浏览器。如果您想使用更新版本的 `pdfjs-dist` 或者想使用自定义构建的 `pdfjs-dist`，可以通过提供一个自定义的 `pdfjs` 函数来实现，该函数返回一个 Promise，解析为 `PDFJS` 对象。

在下面的示例中，我们使用了 `pdfjs-dist` 的“legacy”（见 [pdfjs 文档](https://github.com/mozilla/pdf.js/wiki/Frequently-Asked-Questions#which-browsersenvironments-are-supported)）构建版本，其中包含了一些默认构建中未包含的 polyfill。

```{=mdx}
<Npm2Yarn>
  pdfjs-dist
</Npm2Yarn>

```

In [ ]:
import { WebPDFLoader } from "@langchain/community/document_loaders/web/pdf";

const blob = new Blob(); // e.g. from a file input

const customBuildLoader = new WebPDFLoader(blob, {
  // you may need to add `.then(m => m.default)` to the end of the import
  // @lc-ts-ignore
  pdfjs: () => import("pdfjs-dist/legacy/build/pdf.js"),
});

## 消除多余的空格

PDF 有多种类型，这使得读取它们变得具有挑战性。加载器会解析各个文本元素，并默认用空格将它们连接在一起，但
如果你看到过多的空格，这可能不是期望的行为。在这种情况下，你可以像这样使用空字符串覆盖分隔符：

In [ ]:
import { WebPDFLoader } from "@langchain/community/document_loaders/web/pdf";

// new Blob(); e.g. from a file input
const eliminatingExtraSpacesLoader = new WebPDFLoader(new Blob(), {
  parsedItemSeparator: "",
});

## API 参考文档

如需详细了解 WebPDFLoader 的所有功能和配置，请访问 API 参考文档：https://api.js.langchain.com/classes/langchain_community_document_loaders_web_pdf.WebPDFLoader.html